In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [3]:
load_dotenv()

llm = ChatOpenAI()

In [4]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [5]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [6]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [7]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [8]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza go to the doctor?\n\nBecause it was feeling saucy!',
 'explanation': 'This joke plays on the double meaning of the word "saucy." In one context, "saucy" can mean bold or cheeky in behavior. In the context of the joke, "saucy" refers to the sauce on the pizza. The punchline suggests that the pizza went to the doctor because it was feeling "saucy" in the sense of being covered in sauce. This humorously personifies the pizza and adds a playful twist to the idea of a food item seeking medical attention.'}

In [9]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the doctor?\n\nBecause it was feeling saucy!', 'explanation': 'This joke plays on the double meaning of the word "saucy." In one context, "saucy" can mean bold or cheeky in behavior. In the context of the joke, "saucy" refers to the sauce on the pizza. The punchline suggests that the pizza went to the doctor because it was feeling "saucy" in the sense of being covered in sauce. This humorously personifies the pizza and adds a playful twist to the idea of a food item seeking medical attention.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09b8a6-5115-6314-8002-e3fec7bdbe3b'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-27T10:11:55.191091+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09b8a6-3fb9-69bc-8001-b4cde49c96f3'}}, tasks=(), interrupts=())